In [64]:
from sklearn.linear_model import LogisticRegression
import numpy as np
import pandas as pd
clf=LogisticRegression()
import matplotlib.pyplot as plt

In [65]:
train=pd.read_csv("training_titanic.csv")
test=pd.read_csv("titanic.csv")

In [66]:
train.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,2,"Weisz, Mrs. Leopold (Mathilde Francoise Pede)",female,29.0,1,0,228414,26.000,NaN,S,1
1,3,"Williams, Mr. Howard Hugh ""Harry""",male,NaN,0,0,A/5 2466,8.050,NaN,S,0
2,2,"Morley, Mr. Henry Samuel (""Mr Henry Marshall"")",male,39.0,0,0,250655,26.000,NaN,S,0
3,3,"Palsson, Mrs. Nils (Alma Cornelia Berglund)",female,29.0,0,4,349909,21.075,NaN,S,0
4,3,"Sutehall, Mr. Henry Jr",male,25.0,0,0,SOTON/OQ 392076,7.050,NaN,S,0


In [67]:
train.isnull().sum()

Pclass        0
Name          0
Sex           0
Age         132
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       514
Embarked      1
Survived      0
dtype: int64

In [68]:
train_data = train.copy()
train_data["Age"].fillna(train["Age"].median(skipna=True), inplace=True)
train_data["Embarked"].fillna(train['Embarked'].value_counts().idxmax(), inplace=True)
train_data.drop('Cabin', axis=1, inplace=True)


In [69]:
train_data['TravelAlone']=np.where((train_data["SibSp"]+train_data["Parch"])>0, 0, 1)
train_data.drop('SibSp', axis=1, inplace=True)
train_data.drop('Parch', axis=1, inplace=True)

In [70]:
training=pd.get_dummies(train_data, columns=["Pclass","Embarked","Sex"])
training.drop('Sex_female', axis=1, inplace=True)

final_train = training
final_train.head()


,Name,Age,Ticket,Fare,Survived,TravelAlone,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S,Sex_male
0,"Weisz, Mrs. Leopold (Mathilde Francoise Pede)",29.0,228414,26.000,1,0,0,1,0,0,0,1,0
1,"Williams, Mr. Howard Hugh ""Harry""",29.0,A/5 2466,8.050,0,1,0,0,1,0,0,1,1
2,"Morley, Mr. Henry Samuel (""Mr Henry Marshall"")",39.0,250655,26.000,0,1,0,1,0,0,0,1,1
3,"Palsson, Mrs. Nils (Alma Cornelia Berglund)",29.0,349909,21.075,0,0,0,0,1,0,0,1,0
4,"Sutehall, Mr. Henry Jr",25.0,SOTON/OQ 392076,7.050,0,1,0,0,1,0,0,1,1


In [71]:
test_data = test.copy()
test_data["Age"].fillna(train["Age"].median(skipna=True), inplace=True)
test_data["Fare"].fillna(train["Fare"].median(skipna=True), inplace=True)
test_data.drop('Cabin', axis=1, inplace=True)

test_data['TravelAlone']=np.where((test_data["SibSp"]+test_data["Parch"])>0, 0, 1)

test_data.drop('SibSp', axis=1, inplace=True)
test_data.drop('Parch', axis=1, inplace=True)

testing = pd.get_dummies(test_data, columns=["Pclass","Embarked","Sex"])

testing.drop('Sex_female', axis=1, inplace=True)
#testing.drop('PassengerId', inplace=True)
#testing.drop('Name', axis=1, inplace=True)
#testing.drop('Ticket', axis=1, inplace=True)

final_test = testing
final_test.head()

,Name,Age,Ticket,Fare,TravelAlone,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S,Sex_male
0,"Davies, Master. John Morgan Jr",8.0,C.A. 33112,36.7500,0,0,1,0,0,0,1,1
1,"Leader, Dr. Alice (Farnham)",49.0,17465,25.9292,1,1,0,0,0,0,1,0
2,"Kilgannon, Mr. Thomas J",29.0,36865,7.7375,1,0,0,1,0,1,0,1
3,"Jacobsohn, Mrs. Sidney Samuel (Amy Frances Chr...",24.0,243847,27.0000,0,0,1,0,0,0,1,0
4,"McGough, Mr. James Robert",36.0,PC 17473,26.2875,1,1,0,0,0,0,1,1


In [72]:
cols=["Age","Fare","Sex_male","TravelAlone"]
x=final_train[cols]
y=final_train.Survived
z=final_test[cols]

In [73]:
clf.fit(x,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [74]:
clf.score(x,y)

0.7694610778443114

In [75]:
pred=clf.predict(z)

In [76]:
pro=clf.predict(x)

In [77]:
pro.shape

(668,)

In [79]:
from sklearn.metrics import classification_report
print(classification_report(pro,y))

             precision    recall  f1-score   support

          0       0.83      0.79      0.81       419
          1       0.68      0.73      0.70       249

avg / total       0.77      0.77      0.77       668



In [80]:
from sklearn.metrics import confusion_matrix
confusion_matrix(pro,y)

array([[332,  87],
       [ 67, 182]], dtype=int64)

In [82]:
from sklearn.metrics import classification_report
print(classification_report(y,pro))

             precision    recall  f1-score   support

          0       0.79      0.83      0.81       399
          1       0.73      0.68      0.70       269

avg / total       0.77      0.77      0.77       668

